In [ ]:
import os
from dotenv import load_dotenv
from openai import OpenAI
from src.web_tasks.single_page_summary import WebsiteContentSummarizer
from rich.markdown import Markdown
from rich.console import Console

In [3]:
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

In [4]:
openai = OpenAI()
openai_model = "gpt-4o-mini"

In [5]:
summariser = WebsiteContentSummarizer("https://dirttrackriders.co.uk/about",openai,openai_model)

system_prompt = "You are an assistant that analyses the contents of a website\n \
and provides a comprehensive summary, ignoring text that might be navigation related. \
Respond in markdown."

user_prompt = ("The contents of this website is as follows;\n"
                "please provide a short summary of this website in markdown.\n"
                "If it includes news or announcements, then summarize these too. \n\n"
                "Finish the response with an encouraging message to relevant audience on why they should check out the website.\n" )


In [6]:
summary = summariser.summarize(system_prompt, user_prompt)
console = Console()
console.print(Markdown(summary))

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                        Summary of 'Learn about Dirt Track Racing in the UK. Flat Track'                         ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛

The website provides comprehensive information about flat track racing in the UK, focusing on the Dirt Track Riders
Association (DTRA), which has been organizing championships since 2005. The UK flat track scene is described as    
vibrant and growing, with an increasing number of new riders and bikes entering the sport each year. Races         
primarily occur on speedway tracks, featuring 12 riders in three lines, with races lasting between six to fifteen  
laps.                                                                                                              

The DTRA caters to racers of different levels, particularly highlighting their Pro class, which features top UK    
riders who compete at international standards. The site emphasizes affordability, as club membership and race entry
costs are kept low to encourage broader participation. It also notes that many riders can race an entire season    
using only one rear tyre, minimizing expenses.                                                                     

The community aspect of the DTRA is underscored through testimonials from current members who praise the fun,      
friendships, and skills gained through participation.                                                              


                                                 Encouraging Note                                                  

If you have an interest in motorcycle racing or are looking to get involved, this website is an excellent resource 
to learn about the exciting world of dirt track racing in the UK. Whether you're a potential rider or a spectator, 
there's plenty of information to help you get started and enjoy this thrilling sport! Check it out and join the    
community!